In [1]:
from datetime import datetime
from itertools import combinations
from IPython.display import display, HTML

import pandas

import core

In [2]:
tickers = core.get_tickers("tickers.xml")

names = [t.name for t in tickers]
symbs = [t.symbol for t in tickers]

In [3]:
start_date = datetime(2012, 5, 1)
end_date = datetime(2022, 5, 1)

In [4]:
ticker_datas = {
    s: core.download_daily_avg(s, start_date, end_date) for s in symbs
}

In [5]:
# Correlations between same tickers is one
corrs = {(t, t): 1 for t in symbs}

# Iterate over unique pairs of tickers
for a, b in combinations(symbs, 2):
    # Count correletion between tickers dataframes
    pair_df = [ticker_datas[a], ticker_datas[b]]
    df = pandas.concat(pair_df, axis=1, join="inner")
    df_corr = df.corr(method="pearson")
    # Store correlations in dictinary
    a_key, b_key = df_corr.columns.tolist()
    pair = tuple(sorted([a, b]))
    corrs[pair] = round(df_corr[a_key][b_key], 5)

In [6]:
# Table of correlations
table = pandas.DataFrame(index=names, columns=names)

# Ticker's symbol to ticker's name dictinary
symb_name = {t.symbol: t.name for t in tickers}

# Store correlations to suitable table
for col in symbs:
    for row in symbs:
        k = tuple(sorted([col, row]))
        table[symb_name[row]][symb_name[col]] = corrs[k]

# Output table
html_fmted = core.make_html("html/dataframe.html", table.to_html())
display(HTML(html_fmted))

,S&P500,Nasdaq,Dow Jones,Alphabet,Amazon,Tesla,Meta,Bitcoin,Ethereum,Binance,XRP,Cardano,Solana,Litecoin
S&P500,1,0.99118,0.98808,0.98303,0.9571,0.86838,0.95558,0.91721,0.8988,0.90034,0.45094,0.83667,0.78517,0.75122
Nasdaq,0.99118,1,0.971,0.98167,0.97544,0.89463,0.95787,0.91858,0.85438,0.85212,0.40107,0.81068,0.72534,0.73805
Dow Jones,0.98808,0.971,1,0.95436,0.95222,0.7971,0.95483,0.86953,0.88662,0.89504,0.48412,0.84083,0.71615,0.78528
Alphabet,0.98303,0.98167,0.95436,1,0.93742,0.9121,0.92848,0.93307,0.93225,0.92691,0.48473,0.87658,0.83634,0.72681
Amazon,0.9571,0.97544,0.95222,0.93742,1,0.82228,0.94964,0.82339,0.697,0.71373,0.2465,0.66409,0.40768,0.697
Tesla,0.86838,0.89463,0.7971,0.9121,0.82228,1,0.76725,0.92954,0.88754,0.86129,0.4313,0.77718,0.7719,0.63248
Meta,0.95558,0.95787,0.95483,0.92848,0.94964,0.76725,1,0.83611,0.74763,0.74278,0.40799,0.79406,0.41908,0.75039
Bitcoin,0.91721,0.91858,0.86953,0.93307,0.82339,0.92954,0.83611,1,0.92708,0.92333,0.57048,0.88917,0.67595,0.8078
Ethereum,0.8988,0.85438,0.88662,0.93225,0.697,0.88754,0.74763,0.92708,1,0.96542,0.65681,0.92711,0.88201,0.67407
Binance,0.90034,0.85212,0.89504,0.92691,0.71373,0.86129,0.74278,0.92333,0.96542,1,0.59801,0.90022,0.8082,0.62061
